# fit central density and temperature to the given mass of the star 

Date of creation: 13.07.2020

Last updated: 13.07.2020

In [1]:
import math
import numpy as np
import pandas as pd

%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
import NotebookLoader
import Physics as ph
import SSM18 as s
import Rmodel as r

importing Jupyter notebook from Physics.ipynb
Physics version 1.14 13.07.2020
importing Jupyter notebook from SSM18.ipynb
SSM18 version 1.9 13.07.2020
importing Jupyter notebook from Rmodel.ipynb
Rmodel version 1.05 16.07.2020


In [3]:
def load_hydrogen():
    filename = 'Mass/hydrogen.csv'
    return pd.read_csv(filename)
epoch = 0
dfx = load_hydrogen()
dimM = len(dfx.index)
dfx

,Unnamed: 0,Epoch00,Epoch01
0,0,0.732,0.648658
1,1,0.732,0.652529
2,2,0.732,0.654702
3,3,0.732,0.656470
4,4,0.732,0.658008
...,...,...,...
996,996,0.732,0.732000
997,997,0.732,0.732000
998,998,0.732,0.732000
999,999,0.732,0.732000


In [4]:
epoch = 0
hyd = dfx['Epoch%02d' % epoch].values

In [5]:
"""
m is M/s.M (must be between 0 and 1)
"""
def XYZ(m, Z=0.02):
    i = int(m*(len(hyd) - 1))
    if i > len(hyd) - 1:
        i = len(hyd) - 1
    X = hyd[i]
    Y = 1 - Z - X
    return (X, Y, Z)
    

In [6]:
XYZ(0)

(0.732, 0.248, 0.02)

In [7]:
%%time
t0=1.33300e+07 
d0=94.5544713300
n, M, R, L, T, df = r.model_R(d0, t0, XYZ, debug=False)
n, M/s.M, R/s.R, L/s.L, T

Wall time: 638 ms


(10296,
 0.7102542200572106,
 0.398559027824778,
 0.5212346685697263,
 7771.602719627941)

### Поиск комбинации центральных плотности и температуры, дающих решение с массой равной солнечной. (Первый вариант)

Для интегрирования уравнений из центра надо задать плотность и температуру в центре, но только одна их комбинации 
соответствует решению с массой, равной солнечной (при фиксированном хим. составе)

Внешний цикл по температурам, внутренний цикл - по плотности.
При фиксированной температуре постепенно повышаем плотность, пока не появится стабильное решение с массой < 1.
После этого возвращаемся на 1 шаг назад по плотности, уменьшаем шаг по плотности в 10 раз и повторям процедуру.
Так делаем пока шаг по плотности не станет меньше 1e-8.

In [8]:
%%time
dim = 11
for j in range(dim) :
#     T0 = 1.333e7 + 0.001e7*j  # M = M_sol, epoch = 0
    T0 = 14.51e6 + 0.01e6*j
    step = 10 # step by density
    exc = True
    den0 = 60
    if j > 0:
        df_prev = df.copy()
    while step > 1e-8 :    
        for i in range(10) :
#             print('j=%d i=%d step=%.9f d=%.8f t=%.0f' % (j, i, step, den0, T0))
            try :
                n, M, R, L, T, df = r.model_R(den0, T0, XYZ, debug=False)
            except Exception as error:
#                 print(T0, den0, 'Caught this error: ' + repr(error))
                exc = True
                den0 += step
                break
            if exc :
                exc = False
                print('\tT0=%.5e D0=%.10f n=%d M=%.4f R=%.4f L=%.4f T=%.0f' % (T0, den0, n, M/s.M, R/s.R, L/s.L, T))
                den0 -= step
                step /= 10.
                break
            den0 -= step
    print('# T0=%.5e D0=%.2f n=%d M=%.4f R=%.4f L=%.3f T=%.0f' % (T0, den0, n, M/s.M, R/s.R, L/s.L, T))
    if M/s.M > 1 :
#         df = df_prev
        break
print("Done!")

	T0=1.45100e+07 D0=100.0000000000 n=10276 M=0.7844 R=0.4043 L=0.8849 T=8808
	T0=1.45100e+07 D0=91.0000000000 n=10479 M=0.8359 R=0.4290 L=0.8644 T=8505
	T0=1.45100e+07 D0=90.1000000000 n=11022 M=0.9426 R=0.5000 L=0.9217 T=7999
	T0=1.45100e+07 D0=90.0800000000 n=11259 M=0.9671 R=0.5336 L=0.9245 T=7753
	T0=1.45100e+07 D0=90.0740000000 n=11805 M=0.9950 R=0.6143 L=0.9254 T=7228
	T0=1.45100e+07 D0=90.0739000000 n=11957 M=0.9980 R=0.6347 L=0.9254 T=7106
	T0=1.45100e+07 D0=90.0738600000 n=12185 M=1.0008 R=0.6713 L=0.9254 T=6913
	T0=1.45100e+07 D0=90.0738600000 n=12185 M=1.0008 R=0.6713 L=0.9254 T=6913
	T0=1.45100e+07 D0=90.0738596000 n=12203 M=1.0010 R=0.6772 L=0.9254 T=6883
	T0=1.45100e+07 D0=90.0738595600 n=12203 M=1.0010 R=0.6821 L=0.9254 T=6857
# T0=1.45100e+07 D0=90.07 n=12203 M=1.0010 R=0.6821 L=0.925 T=6857
Done!
Wall time: 1min 28s


In [9]:
# df = df_prev
df

,Mass,Radius,Temperature,Density,Luminosity,Hydrogen,Opacity,Convection,Pressure,Velocity
0,0.000000e+00,0.000000e+00,1.451000e+07,90.073860,0.000000e+00,0.732,1.445415,0.0,1.794120e+17,1.208753e-10
1,1.272082e+23,6.960000e+06,1.451000e+07,90.073860,2.198077e+24,0.732,1.445415,0.0,1.794120e+17,1.208753e-10
2,1.017666e+24,1.392000e+07,1.450999e+07,90.073848,1.758460e+25,0.732,1.445416,0.0,1.794119e+17,1.208749e-10
3,3.434620e+24,2.088000e+07,1.450999e+07,90.073782,5.934786e+25,0.732,1.445417,0.0,1.794117e+17,1.208745e-10
4,8.141317e+24,2.784000e+07,1.450997e+07,90.073704,1.406759e+26,0.732,1.445420,0.0,1.794113e+17,1.208738e-10
...,...,...,...,...,...,...,...,...,...,...
12198,1.992041e+33,4.747615e+10,6.885800e+03,0.000085,3.562822e+33,0.732,0.401010,1.0,7.997807e+07,1.533343e-47
12199,1.992041e+33,4.747619e+10,6.880123e+03,0.000085,3.562822e+33,0.732,0.401009,1.0,7.981326e+07,1.518116e-47
12200,1.992041e+33,4.747622e+10,6.874448e+03,0.000084,3.562822e+33,0.732,0.401008,1.0,7.964866e+07,1.503031e-47
12201,1.992041e+33,4.747625e+10,6.868771e+03,0.000084,3.562822e+33,0.732,0.401006,1.0,7.948425e+07,1.488080e-47


In [10]:
epoch = 0
_ = df.to_csv(line_terminator='\n')
filename = "FromCenter/3epoch%.02d.csv" % epoch
print(filename)
csv_file = open(filename, "wt")
n = csv_file.write(_)
csv_file.close()

FromCenter/3epoch00.csv


### Поиск комбинации центральных плотности и температуры, дающих решение с массой равной солнечной. (Второй вариант)

Для интегрирования уравнений из центра надо задать плотность и температуру в центре, но только одна их комбинации соответствует решению с массой, равной солнечной (при фиксированном хим. составе)

Альтернативный поиск (в обратном порядке)

Внешний цикл по плотности, внутренний цикл - по температуре. При фиксированной плотности постепенно понижаем температуру, пока не появится стабильное решение с массой < 1. После этого возвращаемся на 1 шаг назад по температуре, уменьшаем шаг по температуре в 10 раз и повторям процедуру. Так делаем пока шаг по температуре не станет меньше 0.01 градуса.

In [ ]:
%%time
# Another order of fit - density outside
dim = 11
for j in range(dim) :
    den0 = 94 + 0.1*j # 94.4 + 0.1*j
    step = 1e6 # step for Temperature
    exc = True
    T0 = 15
    if j > 0:
        df_prev = df.copy()
    while step > 1e-2 :    
        for i in range(10) :
            try :
                n, M, R, L, T, df = model_R(den0, T0, debug=False)
            except Exception as error:
#                 print(T0, den0, 'Caught this error: ' + repr(error))
                exc = True
                T0 -= step
                break
            if exc :
                exc = False
                print('\tT0=%.5f D0=%.10f n=%d M=%.4f R=%.4f L=%.4f T=%.0f' % (T0, den0, n, M/M_sol, R/R_sol, L/L_sol, T))
                T0 += step
                step /= 10.
                break
            
    print('# T0=%.5f D0=%.2f n=%d M=%.4f R=%.4f L=%.3f T=%.0f' % (T0, den0, n, M/M_sol, R/R_sol, L/L_sol, T))
    if M/M_sol > 1. :
#         df = df_prev
        break
print("Done!")

In [ ]:
# df = df_prev
df

In [ ]:
_ = df.to_csv(line_terminator='\n')
filename = "FromCenter/2epoch%.02d.csv" % epoch
print(filename)
csv_file = open(filename, "wt")
n = csv_file.write(_)
csv_file.close()

In [30]:
T0=1.33000e+07 
den0=94.4000000000
n, M, R, L, T, df = model_R(den0, T0, debug=False)
n, M/M_sol, R/R_sol, L/L_sol, T

(11049,
 0.8772552888605576,
 0.5009900889800694,
 0.6252592215835585,
 7252.863270233664)